In [1]:
!pip install opencv-python dlib


DEPRECATION: Loading egg at c:\project\deep\lib\site-packages\radam-0.0.1-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
!pip install opencv-python dlib imutils

DEPRECATION: Loading egg at c:\project\deep\lib\site-packages\radam-0.0.1-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
1. 카메라 행렬 설정
카메라 행렬(camera_matrix)의 값이 실제 카메라의 초점 거리와 센서 크기에 맞지 않으면 정확한 포즈 추정이 어렵습니다. 현재는 화면의 가로 크기를 초점 거리로 사용하고 있지만, 실제 카메라의 매개변수를 사용해야 합니다.

2. 왜곡 계수
왜곡 계수(dist_coeffs)를 0으로 설정했는데, 실제 카메라의 왜곡 계수를 고려해야 할 수 있습니다. 실제 왜곡 계수를 모르더라도, 0으로 설정하는 것은 기본값입니다.

3. 오일러 각도 추출
cv2.decomposeProjectionMatrix를 사용하여 오일러 각도를 추출하는 방법은 좀 복잡할 수 있습니다. 오일러 각도를 직접 계산하거나 다른 방법으로 포즈를 추정할 수 있습니다.

4. PnP 알고리즘의 정확성
cv2.solvePnP의 결과가 정확하지 않을 수 있습니다. 모델 포인트와 이미지 포인트의 매칭이 잘못되었을 수 있습니다.

아래는 개선된 코드로, 카메라 매트릭스와 왜곡 계수를 좀 더 정확하게 설정하고, 회전 벡터에서 직접 피치와 요 각도를 추정하는 방법을 사용합니다.

In [36]:
!pip install torch torchvision transformers


DEPRECATION: Loading egg at c:\project\deep\lib\site-packages\radam-0.0.1-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install opencv-python dlib


DEPRECATION: Loading egg at c:\project\deep\lib\site-packages\radam-0.0.1-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import cv2
import mediapipe as mp
import numpy as np

# MediaPipe Hands 모듈 초기화
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(min_detection_confidence=0.7, min_tracking_confidence=0.7)
mp_drawing = mp.solutions.drawing_utils

def is_palm_facing_camera(hand_landmarks):
    """손바닥이 카메라를 향하고 있는지 확인하는 함수"""
    wrist = np.array([hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].x,
                      hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].y,
                      hand_landmarks.landmark[mp_hands.HandLandmark.WRIST].z])
    
    index_finger_tip = np.array([hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x,
                                 hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y,
                                 hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].z])
    
    thumb_tip = np.array([hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].x,
                          hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].y,
                          hand_landmarks.landmark[mp_hands.HandLandmark.THUMB_TIP].z])
    
    # 손목과 손끝 사이의 벡터를 계산
    wrist_to_index = index_finger_tip - wrist
    wrist_to_thumb = thumb_tip - wrist
    
    # 벡터의 내적을 사용하여 손바닥의 방향 판단
    palm_normal_vector = np.cross(wrist_to_index, wrist_to_thumb)
    
    # 벡터의 z값을 확인 (손바닥이 카메라를 향할 때 z값이 양수임)
    return palm_normal_vector[2] > 0

# 카메라 캡처
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # BGR 이미지를 RGB로 변환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # MediaPipe로 손 랜드마크 검출
    result = hands.process(rgb_frame)

    hand_detected = False

    if result.multi_hand_landmarks:
        for hand_landmarks in result.multi_hand_landmarks:
            hand_detected = True

            # 손바닥이 카메라를 향하고 있는지 판단
            if is_palm_facing_camera(hand_landmarks):
                color = (0, 255, 0)  # 초록색 (정상)
                status = "NORMAL"
            else:
                color = (0, 0, 255)  # 빨간색 (비정상)
                status = "DANGER"

            # 손 랜드마크를 이미지에 그림
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS,
                                      mp_drawing.DrawingSpec(color=color, thickness=2, circle_radius=2),
                                      mp_drawing.DrawingSpec(color=color, thickness=2, circle_radius=2))

            # 바운딩 박스 그리기
            h, w, c = frame.shape
            x_min = min([landmark.x for landmark in hand_landmarks.landmark]) * w
            y_min = min([landmark.y for landmark in hand_landmarks.landmark]) * h
            x_max = max([landmark.x for landmark in hand_landmarks.landmark]) * w
            y_max = max([landmark.y for landmark in hand_landmarks.landmark]) * h
            cv2.rectangle(frame, (int(x_min), int(y_min)), (int(x_max), int(y_max)), color, 2)

            # 상태 텍스트 표시
            cv2.putText(frame, status, (int(x_min), int(y_min) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    if not hand_detected:
        # 손이 검출되지 않은 경우 메시지 표시하지 않음
        pass

    cv2.imshow("Frame", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


c:\project\deep\Lib\site-packages\google\protobuf\symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [3]:
!pip install --upgrade imutils

DEPRECATION: Loading egg at c:\project\deep\lib\site-packages\radam-0.0.1-py3.12.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330

[notice] A new release of pip is available: 24.1.2 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
# USAGE
# python detect_drowsiness.py --shape-predictor shape_predictor_68_face_landmarks.dat
# python detect_drowsiness.py --shape-predictor shape_predictor_68_face_landmarks.dat --alarm alarm.wav

# import the necessary packages
from scipy.spatial import distance as dist
from imutils.video import VideoStream
from imutils import face_utils
from threading import Thread
import numpy as np
# import playsound
import argparse
import imutils
import time
import dlib
import cv2

# def sound_alarm(path):
# 	# play an alarm sound
# 	playsound.playsound(path)

def eye_aspect_ratio(eye):
	# compute the euclidean distances between the two sets of
	# vertical eye landmarks (x, y)-coordinates
	A = dist.euclidean(eye[1], eye[5])
	B = dist.euclidean(eye[2], eye[4])

	# compute the euclidean distance between the horizontal
	# eye landmark (x, y)-coordinates
	C = dist.euclidean(eye[0], eye[3])

	# compute the eye aspect ratio
	ear = (A + B) / (2.0 * C)

	# return the eye aspect ratio
	return ear
 
# construct the argument parse and parse the arguments
# ap = argparse.ArgumentParser()
# ap.add_argument("-p", "--shape-predictor", required=True,
# 	help="path to facial landmark predictor")
# ap.add_argument("-a", "--alarm", type=str, default="",
# 	help="path alarm .WAV file")
# ap.add_argument("-w", "--webcam", type=int, default=0,
# 	help="index of webcam on system")
# args = vars(ap.parse_args())
 
# define two constants, one for the eye aspect ratio to indicate
# blink and then a second constant for the number of consecutive
# frames the eye must be below the threshold for to set off the
# alarm
EYE_AR_THRESH = 0.3
EYE_AR_CONSEC_FRAMES = 48

# initialize the frame counter as well as a boolean used to
# indicate if the alarm is going off
COUNTER = 0
ALARM_ON = False

# initialize dlib's face detector (HOG-based) and then create
# the facial landmark predictor
print("[INFO] loading facial landmark predictor...")
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

# grab the indexes of the facial landmarks for the left and
# right eye, respectively
(lStart, lEnd) = face_utils.FACIAL_LANDMARKS_IDXS["left_eye"]
(rStart, rEnd) = face_utils.FACIAL_LANDMARKS_IDXS["right_eye"]

# start the video stream thread
print("[INFO] starting video stream thread...")
vs = VideoStream(0).start()
time.sleep(1.0)

# loop over frames from the video stream
count = 0
while True:
	# grab the frame from the threaded video file stream, resize
	# it, and convert it to grayscale
	# channels)
	frame = vs.read()
	frame = imutils.resize(frame, width=450)
	gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

	# detect faces in the grayscale frame
	rects = detector(gray, 0)
	if len(rects) == 0:
		count += 1
		if count >= 30:
			print('Alert, concentrate on driving !!!')
	else:
		count = 0

	# loop over the face detections
	for rect in rects:
		# determine the facial landmarks for the face region, then
		# convert the facial landmark (x, y)-coordinates to a NumPy
		# array
		shape = predictor(gray, rect)
		shape = face_utils.shape_to_np(shape)

		# extract the left and right eye coordinates, then use the
		# coordinates to compute the eye aspect ratio for both eyes
		leftEye = shape[lStart:lEnd]
		rightEye = shape[rStart:rEnd]
		leftEAR = eye_aspect_ratio(leftEye)
		rightEAR = eye_aspect_ratio(rightEye)

		# average the eye aspect ratio together for both eyes
		ear = (leftEAR + rightEAR) / 2.0
		# print(ear.dtype)

		# compute the convex hull for the left and right eye, then
		# visualize each of the eyes
		leftEyeHull = cv2.convexHull(leftEye)
		rightEyeHull = cv2.convexHull(rightEye)
		cv2.drawContours(frame, [leftEyeHull], -1, (0, 255, 0), 1)
		cv2.drawContours(frame, [rightEyeHull], -1, (0, 255, 0), 1)

		# check to see if the eye aspect ratio is below the blink
		# threshold, and if so, increment the blink frame counter
		if ear < EYE_AR_THRESH:
			COUNTER += 1

			# if the eyes were closed for a sufficient number of
			# then sound the alarm
			if COUNTER >= EYE_AR_CONSEC_FRAMES:
				# if the alarm is not on, turn it on
				if not ALARM_ON:
					ALARM_ON = True

					# check to see if an alarm file was supplied,
					# and if so, start a thread to have the alarm
					# sound played in the background
					# if args["alarm"] != "":
					# 	t = Thread(target=sound_alarm,
					# 		args=(args["alarm"],))
					# 	t.deamon = True
					# 	t.start()

				# draw an alarm on the frame
				cv2.putText(frame, "DROWSINESS ALERT!", (10, 30),
					cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)

		# otherwise, the eye aspect ratio is not below the blink
		# threshold, so reset the counter and alarm
		else:
			COUNTER = 0
			ALARM_ON = False

		# draw the computed eye aspect ratio on the frame to help
		# with debugging and setting the correct eye aspect ratio
		# thresholds and frame counters
		# cv2.putText(frame, "EAR: {:.2f}".format(ear), (300, 30),
		# 	cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
 
	# show the frame
	cv2.imshow("Frame", frame)
	key = cv2.waitKey(1) & 0xFF
 
	# if the `q` key was pressed, break from the loop
	if key == ord("q"):
		break

# do a bit of cleanup
cv2.destroyAllWindows()
vs.stop()

[INFO] loading facial landmark predictor...
[INFO] starting video stream thread...
Alert, concentrate on driving !!!
Alert, concentrate on driving !!!
Alert, concentrate on driving !!!
Alert, concentrate on driving !!!
Alert, concentrate on driving !!!
Alert, concentrate on driving !!!
Alert, concentrate on driving !!!
Alert, concentrate on driving !!!
Alert, concentrate on driving !!!
Alert, concentrate on driving !!!
Alert, concentrate on driving !!!
Alert, concentrate on driving !!!
Alert, concentrate on driving !!!
Alert, concentrate on driving !!!
Alert, concentrate on driving !!!
Alert, concentrate on driving !!!
Alert, concentrate on driving !!!
Alert, concentrate on driving !!!
Alert, concentrate on driving !!!
Alert, concentrate on driving !!!
Alert, concentrate on driving !!!
Alert, concentrate on driving !!!
Alert, concentrate on driving !!!
Alert, concentrate on driving !!!
Alert, concentrate on driving !!!
Alert, concentrate on driving !!!
Alert, concentrate on driving !!!